# ADS 509 Sentiment Assignment

This notebook holds the Sentiment Assignment for Module 6 in ADS 509, Applied Text Mining. Work through this notebook, writing code and answering questions where required. 

In a previous assignment you put together Twitter data and lyrics data on two artists. In this assignment we apply sentiment analysis to those data sets. If, for some reason, you did not complete that previous assignment, data to use for this assignment can be found in the assignment materials section of Blackboard. 


In [1]:
import os
import re
import emoji
import pandas as pd
import numpy as np

from collections import Counter, defaultdict
from string import punctuation

from nltk.corpus import stopwords

sw = stopwords.words("english")

In [9]:
# Add any additional import statements you need here
import warnings

In [19]:
# change `data_location` to the location of the folder on your machine.
data_location = "/Users/clairebentzen/Desktop/MDAS/ADS 509 - Applied Text Mining/Module 2/Assignment2.1/M1 Results/"

# These subfolders should still work if you correctly stored the 
# data from the Module 1 assignment
twitter_folder = "twitter/"
lyrics_folder = "lyrics/"

# Specify artist_files for twitter data
artist_files = {'cher':'cher_followers_data.txt',
                'robyn':'robynkonichiwa_followers_data.txt'}

positive_words_file = "positive-words.txt"
negative_words_file = "negative-words.txt"
tidy_text_file = "tidytext_sentiments.txt"

## Data Input

Now read in each of the corpora. For the lyrics data, it may be convenient to store the entire contents of the file to make it easier to inspect the titles individually, as you'll do in the last part of the assignment. In the solution, I stored the lyrics data in a dictionary with two dimensions of keys: artist and song. The value was the file contents. A Pandas data frame would work equally well. 

For the Twitter data, we only need the description field for this assignment. Feel free all the descriptions read it into a data structure. In the solution, I stored the descriptions as a dictionary of lists, with the key being the artist. 




In [11]:
# Read in the lyrics data
# Specify pathway to lyrics folder
lyrics_path = data_location + lyrics_folder

# Create a dataframe to store results
lyrics_data = pd.DataFrame(columns=['artist', 'song', 'lyrics'])

# Iterate through each file in the lyrics folder
for artist in os.listdir(lyrics_path):
    artist_path = os.path.join(lyrics_path, artist)
    
    # Iterate through each file in the artist folders
    for song in os.listdir(artist_path):
        song_path = os.path.join(artist_path, song)
        rem_prefix = song.removeprefix(f'{artist}_')
        song_title = rem_prefix.removesuffix('.txt')

        # Open and read the contents of the file (song)
        with open(song_path, 'r') as file:
            contents = file.read()
            # Prepare data to add to dataframe
            data = {'artist': artist, 'song': song_title, 'lyrics': contents}
            # The df.append() function is deprecated, so we will ignore warnings here
            with warnings.catch_warnings():
                warnings.simplefilter('ignore')
                # Append row of data to lyrics_df
                lyrics_data = lyrics_data.append(data, ignore_index=True)

In [15]:
# View lyrics_data dataframe
lyrics_data.head()

,artist,song,lyrics
0,robyn,includemeout,"""Include Me Out""\n\n\n\nIt is really very simp..."
1,robyn,electric,"""Electric""\n\n\n\nElectric...\n\nIt's electric..."
2,robyn,beach2k20,"""Beach 2K20""\n\n\n\n(So you wanna go out?\nHow..."
3,robyn,lovekills,"""Love Kills""\n\n\n\nIf you're looking for love..."
4,robyn,timemachine,"""Time Machine""\n\n\n\nHey, what did I do?\nCan..."


In [21]:
# Read in the twitter data
# Read cher twitter data
twitter_data = pd.read_csv(data_location + twitter_folder + artist_files['cher'],
                           sep="\t",
                           quoting=3)

twitter_data['artist'] = "cher"

# Read robyn twitter data
twitter_data_2 = pd.read_csv(data_location + twitter_folder + artist_files['robyn'],
                             sep="\t",
                             quoting=3)
twitter_data_2['artist'] = "robyn"

# Concat twitter dataframes
twitter_data = pd.concat([
    twitter_data,twitter_data_2])
    
del(twitter_data_2)

In [25]:
# Keep only artist and description columns
twitter_data = twitter_data[['description', 'artist']]

In [67]:
# Read in the positive and negative words and the
# tidytext sentiment. Store these so that the positive
# words are associated with a score of +1 and negative words
# are associated with a score of -1. You can use a dataframe or a 
# dictionary for this.

# Read positive words and add score column
pos_words = pd.read_csv('positive-words.txt', delimiter='\t', skiprows=35, header=None)
pos_words['score'] = 1
pos_words.columns.values[0] = 'word'

# Read negative words and add score column
neg_words = pd.read_csv('negative-words.txt', delimiter='\t', skiprows=35, header=None)
neg_words['score'] = -1
neg_words.columns.values[0] = 'word'

# Read tidytext sentiment
tidytext = pd.read_csv('tidytext_sentiments.txt', delimiter='\t')
tidytext.drop(columns='lexicon', inplace=True)
# Replace sentiment with -1 and 1
tidytext['score'] = tidytext['sentiment'].apply(lambda x: -1 if x == 'negative' else 1)
tidytext.drop(columns='sentiment', inplace=True)

# Combine dataframes
sentiment_scores = pd.concat([pos_words, neg_words, tidytext])
sentiment_scores

,word,score
0,a+,1
1,abound,1
2,abounds,1
3,abundance,1
4,abundant,1
...,...,...
15128,win,1
15129,winner,1
15130,winners,1
15131,winning,1


In [27]:
# View twitter_data dataframe
twitter_data.head()

,description,artist
0,NaN,cher
1,𝙿𝚛𝚘𝚞𝚍 𝚜𝚞𝚙𝚙𝚘𝚛𝚝𝚎𝚛 𝚘𝚏 𝚖𝚎𝚜𝚜𝚢 𝚋𝚞𝚗𝚜 & 𝚕𝚎𝚐𝚐𝚒𝚗𝚐𝚜,cher
2,163㎝／愛かっぷ💜26歳🍒 工〇好きな女の子💓 フォローしてくれたらDMします🧡,cher
3,csu,cher
4,Writer @Washinformer @SpelmanCollege alumna #D...,cher


## Sentiment Analysis on Songs

In this section, score the sentiment for all the songs for both artists in your data set. Score the sentiment by manually calculating the sentiment using the combined lexicons provided in this repository. 

After you have calculated these sentiments, answer the questions at the end of this section.


In [ ]:
# your code here

### Questions

Q: Overall, which artist has the higher average sentiment per song? 

A: <!-- Your answer here -->

---

Q: For your first artist, what are the three songs that have the highest and lowest sentiments? Print the lyrics of those songs to the screen. What do you think is driving the sentiment score? 

A: <!-- Your answer here -->

---

Q: For your second artist, what are the three songs that have the highest and lowest sentiments? Print the lyrics of those songs to the screen. What do you think is driving the sentiment score? 

A: <!-- Your answer here -->

---

Q: Plot the distributions of the sentiment scores for both artists. You can use `seaborn` to plot densities or plot histograms in matplotlib.




## Sentiment Analysis on Twitter Descriptions

In this section, define two sets of emojis you designate as positive and negative. Make sure to have at least 10 emojis per set. You can learn about the most popular emojis on Twitter at [the emojitracker](https://emojitracker.com/). 

Associate your positive emojis with a score of +1, negative with -1. Score the average sentiment of your two artists based on the Twitter descriptions of their followers. The average sentiment can just be the total score divided by number of followers. You do not need to calculate sentiment on non-emoji content for this section.

In [ ]:
# your code here

Q: What is the average sentiment of your two artists? 

A: <!-- Your answer here --> 

---

Q: Which positive emoji is the most popular for each artist? Which negative emoji? 

A: <!-- Your answer here --> 

